In [1]:
import pandas as pd

data = pd.read_excel('nazwa_nowego_pliku.xlsx')
data = data.dropna(subset=['review'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19918 entries, 0 to 19917
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 0.1         19918 non-null  int64         
 1   Unnamed: 0           19918 non-null  int64         
 2   user                 19915 non-null  object        
 3   playtime             19917 non-null  float64       
 4   post_date            19918 non-null  datetime64[ns]
 5   helpfulness          19918 non-null  object        
 6   review               19918 non-null  object        
 7   recommend            19918 non-null  object        
 8   early_access_review  1207 non-null   object        
 9   game_id              19918 non-null  int64         
 10  label                273 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 1.7+ MB


In [126]:
data['playtime'] = data['playtime'].str.replace(',','').astype(float)
data['post_date'] = pd.to_datetime(data['post_date'], format='%B %d, %Y')
data.to_excel('nazwa_nowego_pliku.xlsx')

In [2]:
from sklearn.model_selection import train_test_split

data2 = data.dropna(subset=['label'])
data2['label'] = data2['label'].astype(int)

C:\Users\tymot\AppData\Local\Temp\ipykernel_20308\761110511.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['label'] = data2['label'].astype(int)


In [3]:
# Wybieramy losowe próbki z klasy 0
class_0 = data2[data2['label'] == 0]

# Wybieramy losowe próbki z klasy 1
class_1 = data2[data2['label'] == 1].sample(len(class_0))

# Łączymy wybrane próbki z obu klas w jedną ramkę danych
undersampled_df = pd.concat([class_0, class_1])

data2 = undersampled_df 

In [4]:
X = data2[['review','label']]
X_train, X_test = train_test_split(X,train_size=0.7)

In [5]:
X_train

,review,label
228,cowboy,0
87,PENIS,0
111,shoot things and loot comes outPSA: don't try ...,0
222,Winner Winner Chicken Dinner,0
78,They say alot of swear words,1
...,...,...
200,"Pretty good game,but still early access....be ...",1
172,Holy ♥♥♥♥ what a ♥♥♥♥ show. This game is full ...,1
177,Arthur commits tax evasion on tahiti,1
66,dude its skyrim,0


In [6]:
from datasets import Dataset
aa = Dataset.from_pandas(X_train)
aa = aa.remove_columns(["__index_level_0__"])
aa

Dataset({
    features: ['review', 'label'],
    num_rows: 75
})

In [7]:
a2 = Dataset.from_pandas(X_test)
a2 = a2.remove_columns(["__index_level_0__"])
a2

Dataset({
    features: ['review', 'label'],
    num_rows: 33
})

In [8]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def preprocess_function(examples):
    return tokenizer(examples["review"], truncation=True)

toka = aa.map(preprocess_function, batched=True)
toka2 = a2.map(preprocess_function, batched=True)

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

In [9]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 5
batches_per_epoch = len(toka) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [10]:
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [12]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

tf_train_set = model.prepare_tf_dataset(
    toka,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    toka2,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [13]:
import tensorflow as tf
model.compile(optimizer=optimizer)

from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)



No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [14]:
with tf.device('/CPU:0'):
    model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[metric_callback])

Epoch 1/3
4/4 [==============================] - 49s 11s/step - loss: 0.6704 - val_loss: 0.6575 - accuracy: 0.7576
Epoch 2/3
4/4 [==============================] - 42s 10s/step - loss: 0.6406 - val_loss: 0.6233 - accuracy: 0.7879
Epoch 3/3
4/4 [==============================] - 45s 15s/step - loss: 0.5979 - val_loss: 0.5952 - accuracy: 0.8182


In [19]:

test_texts = "cox"
encoded_test_texts = tokenizer(test_texts, padding=True, truncation=True)

input_ids = np.array(encoded_test_texts['input_ids'])
attention_mask = np.array(encoded_test_texts['attention_mask'])


predictions = model.predict([input_ids, attention_mask])
predicted_probabilities = np.array(predictions.logits)
predicted_probabilities = np.exp(predicted_probabilities) / np.sum(np.exp(predicted_probabilities), axis=1, keepdims=True)

print(f"Text: {test_texts}\nSpam: {predicted_probabilities[0][0]}, Not: {predicted_probabilities[0][1]}\n")


1/1 [==============================] - 0s 52ms/step
Text: cox
Spam: 0.5480402112007141, Not: 0.4519597887992859

